# Denoising AE

In [1]:
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.layers.convolutional import Conv2D, UpSampling2D, MaxPooling2D, Conv2DTranspose
import h5py
import numpy as np
import matplotlib.pyplot as plt
#from pathlib import Path

from keras import backend as keras_backend
keras_backend.set_image_data_format('channels_last')

Using TensorFlow backend.


In [2]:
def get_mnist_samples():
    random_seed = 42
    np.random.seed(random_seed)

    # Read MNIST data. We won't be using the y_train or y_test data
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    pixels_per_image = np.prod(X_train.shape[1:])

    # Cast values into the current floating-point type
    X_train = keras_backend.cast_to_floatx(X_train)
    X_test = keras_backend.cast_to_floatx(X_test)
    
    X_train = np.reshape(X_train, (len(X_train), 28, 28, 1)) 
    X_test = np.reshape(X_test, (len(X_test), 28, 28, 1)) 
    
    # Normalize the range from [0,255] to [0,1]
    X_train /= 255.
    X_test /= 255.

    return (X_train, X_test)

In [6]:
def add_noise_to_mnist(X_train, X_test, noise_factor=0.5): # add noise to the digita
    X_train_noisy = X_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_train.shape) 
    X_test_noisy = X_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_test.shape) 

    X_train_noisy = np.clip(X_train_noisy, 0., 1.)
    X_test_noisy = np.clip(X_test_noisy, 0., 1.)
    return (X_train_noisy, X_test_noisy)

In [7]:
(X_train, X_test) = get_mnist_samples()
(X_train_noisy, X_test_noisy) = add_noise_to_mnist(X_train, X_test, 0.5)

In [ ]:
def draw_noisy_predictions_set(predictions, filename=None):
    plt.figure(figsize=(8, 4))
    for i in range(5):
        plt.subplot(2, 5, i+1)
        plt.imshow(X_test_noisy[i].reshape(28, 28), vmin=0, vmax=1, cmap="gray")
        ax = plt.gca()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        plt.subplot(2, 5, i+6)
        plt.imshow(predictions[i,:,:,0].reshape(28, 28), vmin=0, vmax=1, cmap="gray")
        ax = plt.gca()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.tight_layout()
    file_helper.save_figure(filename+'-predictions')
    plt.show()

## autoencoder 1

In [4]:
# build the autoencoder.
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(28,28,1)))
model.add(MaxPooling2D((2,2,), padding='same'))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D((2,2), padding='same'))
# down to 7, 7, 32 now go back up
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(1, (3,3), activation='sigmoid', padding='same'))
model.compile(optimizer='adadelta', loss='binary_crossentropy')

In [8]:
np.random.seed(42)
history1 = model.fit(X_train_noisy, X_train,
                      epochs=100,
                      batch_size=128,
                      shuffle=True,
                      validation_data=(X_test_noisy, X_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 741s - loss: 0.1938 - val_loss: 0.1311
Epoch 2/100
60000/60000 [==============================] - 327s - loss: 0.1244 - val_loss: 0.1170
Epoch 3/100
60000/60000 [==============================] - 705s - loss: 0.1157 - val_loss: 0.1111
Epoch 4/100
60000/60000 [==============================] - 1485s - loss: 0.1114 - val_loss: 0.1081
Epoch 5/100
60000/60000 [==============================] - 1697s - loss: 0.1086 - val_loss: 0.1055
Epoch 6/100
60000/60000 [==============================] - 1597s - loss: 0.1068 - val_loss: 0.1046
Epoch 7/100
60000/60000 [==============================] - 1485s - loss: 0.1054 - val_loss: 0.1036
Epoch 8/100
60000/60000 [==============================] - 747s - loss: 0.1042 - val_loss: 0.1029
Epoch 9/100
60000/60000 [==============================] - 671s - loss: 0.1034 - val_loss: 0.1028
Epoch 10/100
60000/60000 [==============================] - 305s

In [ ]:
predictions1 = model1.predict(X_test_noisy)
draw_noisy_predictions_set(predictions1)

## Autoencoder 2

In [ ]:
# build the autoencoder.
    model = Sequential()
    model.add(Conv2D(32, (3,3), activation='relu', padding='same', strides=2, input_shape=(28,28,1)))
    model.add(Conv2D(32, (3,3), activation='relu', padding='same', strides=2))
    # down to 7, 7, 32 now go back up
    model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
    model.add(UpSampling2D((2,2)))
    model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
    model.add(UpSampling2D((2,2)))
    model.add(Conv2D(1, (3,3), activation='sigmoid', padding='same'))
    
    model.compile(optimizer='adadelta', loss='binary_crossentropy')

In [ ]:
model2 = build_autoencoder2()
weights_filename = "NB5-Denoising-AE2"
np.random.seed(42)
history2 = model2.fit(X_train_noisy, X_train,
                      epochs=100, 
                      batch_size=128, 
                      shuffle=True, 
                      validation_data=(X_test_noisy, X_test))

In [ ]:
predictions2 = model2.predict(X_test_noisy)
draw_noisy_predictions_set(predictions2)

## Autoencoder 3

In [ ]:
    # build the autoencoder.
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', padding='same', strides=2, input_shape=(28,28,1)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same', strides=2))
# down to 7, 7, 32 now go back up
model.add(Conv2DTranspose(32, (3,3), activation='relu', strides=2, padding='same'))
model.add(Conv2DTranspose(32, (3,3), activation='relu', strides=2, padding='same'))
model.add(Conv2D(1, (3,3), activation='sigmoid', padding='same'))
    
model.compile(optimizer='adadelta', loss='binary_crossentropy')

In [ ]:
model3 = build_autoencoder3()
weights_filename = "NB5-Denoising-AE3"
np.random.seed(42)
history3 = model3.fit(X_train_noisy, X_train,
                        epochs=100,
                        batch_size=128,
                        shuffle=True,
                        validation_data=(X_test_noisy, X_test))

In [ ]:
predictions3 = model3.predict(X_test_noisy)
draw_noisy_predictions_set(predictions)